<h3> Google Colab Setup </h3>


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


<h3>CNN</h3>



In [4]:
import numpy as np
import pandas as pd

data=np.load('/content/drive/My Drive/CS  214/CNN/data.npy')
target=np.load('/content/drive/My Drive/CS  214/CNN/target.npy')

In [5]:
data = np.array(data, dtype="float32")

In [6]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
labels = lb.fit_transform(target)
labels = to_categorical(labels)
labels

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [7]:
from sklearn.model_selection import train_test_split

train_data,test_data,train_target,test_target=train_test_split(data,labels,test_size=0.2)

In [ ]:
train_data.shape

(3066, 224, 224, 3)

In [ ]:
train_target.shape

(3066, 2)

In [ ]:
test_data.shape

(767, 224, 224, 3)

In [ ]:
test_target.shape

(767, 2)

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [9]:
import tensorflow as tf
from tensorflow import keras

<h3>Resnet50V2</h3>

In [10]:
resnet = tf.keras.applications.ResNet50V2(
    input_shape=None, include_top=True, weights=None, input_tensor=None,
    pooling=None, classes=2
)

resnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
with tf.device('/device:GPU:0'):
  resnet.fit(train_data, train_target, validation_data=(test_data,test_target), epochs = 8)

Epoch 1/8
96/96 [==============================] - 18s 184ms/step - loss: 0.5190 - accuracy: 0.8252 - val_loss: 2.3796 - val_accuracy: 0.5385
Epoch 2/8
96/96 [==============================] - 16s 167ms/step - loss: 0.2932 - accuracy: 0.8920 - val_loss: 2.3548 - val_accuracy: 0.5033
Epoch 3/8
96/96 [==============================] - 16s 167ms/step - loss: 0.2352 - accuracy: 0.9139 - val_loss: 1.0932 - val_accuracy: 0.7640
Epoch 4/8
96/96 [==============================] - 16s 168ms/step - loss: 0.2197 - accuracy: 0.9207 - val_loss: 2.1782 - val_accuracy: 0.5932
Epoch 5/8
96/96 [==============================] - 16s 167ms/step - loss: 0.1721 - accuracy: 0.9426 - val_loss: 0.4503 - val_accuracy: 0.8370
Epoch 6/8
96/96 [==============================] - 16s 167ms/step - loss: 0.1529 - accuracy: 0.9455 - val_loss: 0.3897 - val_accuracy: 0.8735
Epoch 7/8
96/96 [==============================] - 16s 167ms/step - loss: 0.1669 - accuracy: 0.9390 - val_loss: 0.9312 - val_accuracy: 0.7445
Epoch 

In [ ]:
test_predict = resnet.predict(test_data)

In [ ]:
df = pd.DataFrame(test_predict)

In [ ]:
def convert_max(row):
    if row[0] < 0.5:
        row[0] = 0
    else:
        row[0] = 1      
    if row[1] < 0.5:
        row[1] = 0
    else:
        row[1] = 1
    return row

In [ ]:
df = df.apply(lambda row: convert_max(row), axis = 1)

In [ ]:
test_target_df = pd.DataFrame(test_target)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(df, test_target_df)

0.803129074315515

<h3>NASNetMobile</h3> 

In [ ]:
nasnet = tf.keras.applications.NASNetMobile(
    input_shape=None, include_top=True, weights=None, input_tensor=None,
    pooling=None, classes=2
)

nasnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
with tf.device('/device:GPU:0'):
  nasnet.fit(train_data, train_target, validation_data=(test_data,test_target),epochs = 38)

Epoch 1/38
96/96 [==============================] - 40s 418ms/step - loss: 0.3487 - accuracy: 0.8728 - val_loss: 7.6585 - val_accuracy: 0.5007
Epoch 2/38
96/96 [==============================] - 37s 390ms/step - loss: 0.1750 - accuracy: 0.9354 - val_loss: 7.6585 - val_accuracy: 0.5007
Epoch 3/38
96/96 [==============================] - 38s 391ms/step - loss: 0.1276 - accuracy: 0.9586 - val_loss: 7.6584 - val_accuracy: 0.5007
Epoch 4/38
96/96 [==============================] - 38s 393ms/step - loss: 0.1225 - accuracy: 0.9573 - val_loss: 7.6585 - val_accuracy: 0.5007
Epoch 5/38
96/96 [==============================] - 37s 390ms/step - loss: 0.1032 - accuracy: 0.9609 - val_loss: 7.6585 - val_accuracy: 0.5007
Epoch 6/38
96/96 [==============================] - 37s 391ms/step - loss: 0.0967 - accuracy: 0.9644 - val_loss: 6.0857 - val_accuracy: 0.5007
Epoch 7/38
96/96 [==============================] - 38s 391ms/step - loss: 0.0814 - accuracy: 0.9726 - val_loss: 4.9662 - val_accuracy: 0.5007

<h3>Xception</h3>

In [12]:
xception = tf.keras.applications.Xception(
    input_shape=None, include_top=True, weights=None, input_tensor=None,
    pooling=None, classes=2
)

xception.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
with tf.device('/device:GPU:0'):
  xception.fit(train_data, train_target, validation_data=(test_data,test_target),epochs = 5)

Epoch 1/5
96/96 [==============================] - 40s 418ms/step - loss: 0.3865 - accuracy: 0.8757 - val_loss: 0.6928 - val_accuracy: 0.5176
Epoch 2/5
96/96 [==============================] - 39s 403ms/step - loss: 0.2010 - accuracy: 0.9273 - val_loss: 0.6897 - val_accuracy: 0.5111
Epoch 3/5
96/96 [==============================] - 39s 403ms/step - loss: 0.1838 - accuracy: 0.9286 - val_loss: 0.6589 - val_accuracy: 0.7262
Epoch 4/5
96/96 [==============================] - 39s 404ms/step - loss: 0.1481 - accuracy: 0.9468 - val_loss: 0.4670 - val_accuracy: 0.9257
Epoch 5/5
96/96 [==============================] - 39s 403ms/step - loss: 0.0982 - accuracy: 0.9641 - val_loss: 0.2525 - val_accuracy: 0.9596
